# IBM Data Science: Capstone Project
The Battle of Neighbourhoods

## Introduction

An upward trend in migration from HK to UK is observed in recent years. London boroughs are one of the popular places where Hong Kongers love to live in. With this relocation, there's a great potential for Asian food restaurants to grow. 

Based on this assumption, if you wish to set up a business in Asian food, which neighborhoods should you choose? 

In this project, I aim to cluster London neighbourhoods based on Asian catering businesses.  Hope this could give you an idea on which places are more suitable to start with the business. 


## Data

The data I will be using is from accessing the Foursquare API. I will be exploring neighborhoods in London, using the explore function to get the most venue categories of Asian catering in each neighborhood, and then use this feature to group the neighborhoods into clusters.

I will use the geopy library to get the latitude and longitude values of London. Then, use a machine learning clustering algorithm, k-means, to do the custering. Finally, I will use the Folium library to colour code the neighborhoods in London based on their emerging clusters.

## Table of Contents

1. Download and Explore Dataset

2. Explore Neighborhoods in London 

3. Gathering Foursquare Data

4. Cluster Neighborhoods

5. Examine Clusters

### 1. Download and Explore Dataset

In [1]:
!pip install -U numpy
!pip install -U pandas
!pip install -U scipy
!pip install -U scikit-learn
!pip install -U imbalanced-learn
!pip install folium
!pip install sklearn
!pip install beautifulsoup4
!pip install lxml
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 15.7 MB 33.9 MB/s eta 0:00:01
ERROR: tensorflow 2.1.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.5.0 which is incompatible.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.ut

In [2]:
!pip install -U geopy


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 112 kB 17.0 MB/s eta 0:00:01
  Attempting uninstall: geopy
    Found existing installation: geopy 2.0.0
    Uninstalling geopy-2.0.0:
      Successfully uninstalled geopy-2.0.0


In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:
#Scraping the London data
#Reading in the table and setting the header to the first row
df = pd.read_html(r'https://en.wikipedia.org/wiki/List_of_areas_of_London', match = 'Location')
df = pd.DataFrame(df[0])
df.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [5]:
#setting the column labels
df.columns = ['location', 'borough', 'posttown', 'postcode', 'dialcode', 'OSgridref']
df.head()

,location,borough,posttown,postcode,dialcode,OSgridref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [6]:
#dropping unnecessary columns
df.drop(['dialcode', 'OSgridref'], axis =1, inplace = True)
df.columns

Index(['location', 'borough', 'posttown', 'postcode'], dtype='object')

In [7]:
#Only selecting the first postal code and first boroughs listed
first_postcode= df.postcode.str.split(",", expand = True)[0]
df['postcode'] = first_postcode

first_borough= df.borough.str.split(",", expand = True)[0]
df['borough'] = first_borough
df.head()

,location,borough,posttown,postcode
0,Abbey Wood,Bexley,LONDON,SE2
1,Acton,Ealing,LONDON,W3
2,Addington,Croydon[8],CROYDON,CR0
3,Addiscombe,Croydon[8],CROYDON,CR0
4,Albany Park,Bexley,"BEXLEY, SIDCUP",DA5


In [8]:
# Cleaning up the borough names to remove the square brackets with regex
df.borough = df.borough.map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))
df

,location,borough,posttown,postcode
0,Abbey Wood,Bexley,LONDON,SE2
1,Acton,Ealing,LONDON,W3
2,Addington,Croydon,CROYDON,CR0
3,Addiscombe,Croydon,CROYDON,CR0
4,Albany Park,Bexley,"BEXLEY, SIDCUP",DA5
5,Aldborough Hatch,Redbridge,ILFORD,IG2
6,Aldgate,City,LONDON,EC3
7,Aldwych,Westminster,LONDON,WC2
8,Alperton,Brent,WEMBLEY,HA0
9,Anerley,Bromley,LONDON,SE20


In [9]:
#only selecting London Neighborhoods and cross referencing that the selection is correct
df = df[df.posttown == "LONDON"]
df.posttown.value_counts()

LONDON    297
Name: posttown, dtype: int64

In [10]:
!pip install -U geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 9.8 MB/s  eta 0:00:01


In [11]:
# install the Geocoder
!pip -q install geocoder
import geocoder
!pip -q install geopy
print('geopy installed...')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
geopy installed...


In [12]:
from geopy.geocoders import arcgis

In [13]:
# Geocoder starts here
# Defining a function to use --> get_latlng()'''
def get_latlng(arcgis_geocoder):
    
    # Initialize the Location (lat. and long.) to "None"
    lat_lng_coords = None
    
    # While loop helps to create a continous run until all the location coordinates are geocoded
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London, United Kingdom'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Geocoder ends here

In [14]:
sample = get_latlng('SW11')
sample

[51.46023000000008, -0.1687014919999683]

In [15]:
#Using comprehension to create the data for our additional columns
post_codes = [get_latlng(p) for p in df['postcode'].to_list()]


In [16]:
#creating the dataframe which we will append to the end of our existing dataframe
df_c = pd.DataFrame(post_codes, columns = ['latitude', 'longitude'])
df_c.head()

,latitude,longitude
0,51.499741,0.124061
1,51.507408,-0.127699
2,51.513145,-0.078733
3,51.514625,-0.114860
4,51.482490,0.119194


In [17]:
df_lon = pd.concat([df, df_c], axis=1)
df_lon

,location,borough,posttown,postcode,latitude,longitude
0,Abbey Wood,Bexley,LONDON,SE2,51.499741,0.124061
1,Acton,Ealing,LONDON,W3,51.507408,-0.127699
2,NaN,NaN,NaN,NaN,51.513145,-0.078733
3,NaN,NaN,NaN,NaN,51.514625,-0.114860
4,NaN,NaN,NaN,NaN,51.482490,0.119194
5,NaN,NaN,NaN,NaN,51.530130,-0.107969
6,Aldgate,City,LONDON,EC3,51.532760,-0.114007
7,Aldwych,Westminster,LONDON,WC2,51.542690,-0.109138
8,NaN,NaN,NaN,NaN,51.446365,-0.137480
9,Anerley,Bromley,LONDON,SE20,51.500212,-0.115440


In [18]:
df_lon = df_lon.dropna()
df_lon

,location,borough,posttown,postcode,latitude,longitude
0,Abbey Wood,Bexley,LONDON,SE2,51.499741,0.124061
1,Acton,Ealing,LONDON,W3,51.507408,-0.127699
6,Aldgate,City,LONDON,EC3,51.532760,-0.114007
7,Aldwych,Westminster,LONDON,WC2,51.542690,-0.109138
9,Anerley,Bromley,LONDON,SE20,51.500212,-0.115440
10,Angel,Islington,LONDON,EC1,51.530130,-0.107969
12,Archway,Islington,LONDON,N19,51.507408,-0.127699
15,Arnos Grove,Enfield,LONDON,N11,51.507408,-0.127699
16,Balham,Wandsworth,LONDON,SW12,51.507408,-0.127699
17,Bankside,Southwark,LONDON,SE1,51.439820,0.000393


In [19]:
df_lon.rename({'location': 'neighborhood'}, axis = 1, inplace = True)
df_lon

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,neighborhood,borough,posttown,postcode,latitude,longitude
0,Abbey Wood,Bexley,LONDON,SE2,51.499741,0.124061
1,Acton,Ealing,LONDON,W3,51.507408,-0.127699
6,Aldgate,City,LONDON,EC3,51.532760,-0.114007
7,Aldwych,Westminster,LONDON,WC2,51.542690,-0.109138
9,Anerley,Bromley,LONDON,SE20,51.500212,-0.115440
10,Angel,Islington,LONDON,EC1,51.530130,-0.107969
12,Archway,Islington,LONDON,N19,51.507408,-0.127699
15,Arnos Grove,Enfield,LONDON,N11,51.507408,-0.127699
16,Balham,Wandsworth,LONDON,SW12,51.507408,-0.127699
17,Bankside,Southwark,LONDON,SE1,51.439820,0.000393


### 2. Explore Neighborhoods in London 

In [20]:
!pip install -U geopy

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Requirement already up-to-date: geopy in /opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages (2.1.0)


In [21]:
!pip install -q geolocator

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [22]:
# importing geopy library
from geopy.geocoders import Nominatim
  
# calling the Nominatim tool
loc = Nominatim(user_agent="GetLoc")

# entering the location name
location = loc.geocode("London, United Kingdom")
location 

Location(London, Greater London, England, United Kingdom, (51.5073219, -0.1276474, 0.0))

In [23]:
# Having a look at London using a folium map

lat_lon = location.latitude
lng_lon = location.longitude
print (lat_lon, lng_lon)

51.5073219 -0.1276474


In [24]:
# create map of using latitude and longitude values
map_lon= folium.Map(location=[lat_lon, lng_lon], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_lon['latitude'], df_lon['longitude'], df_lon['borough'], df_lon['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_opacity=0.5,
        parse_html=False).add_to(map_lon)  
    
map_lon

### 3. Gathering Foursquare Data

In [25]:
neighborhood_name = df_lon.loc[0, 'neighborhood']
neighborhood_latitude = df_lon.loc[0, 'latitude']
neighborhood_longitude = df_lon.loc[0, 'longitude']

LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID='5I3DLW5XVAABDTQF2J2XRTHXWPANUUNZUUJDXXUJWB251M5D'
CLIENT_SECRET='41VU5CVD1NFCUKDDKCOF4UDN0FTFOUGIJ0RJYOWTDTNOOMWC'
VERSION='20180605'

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=5I3DLW5XVAABDTQF2J2XRTHXWPANUUNZUUJDXXUJWB251M5D&client_secret=41VU5CVD1NFCUKDDKCOF4UDN0FTFOUGIJ0RJYOWTDTNOOMWC&v=20180605&ll=51.499741450000045,0.12406135200006929&radius=500&limit=50'

In [26]:
#Sending the GET request to examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60dd90730def806e93c982a1'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Thamesmead East',
  'headerFullLocation': 'Thamesmead East, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 51.50424145450005,
    'lng': 0.13127656708327387},
   'sw': {'lat': 51.49524144550004, 'lng': 0.11684613691686471}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e4a430e1850545dfc0289ae',
       'name': 'Southmere Lake',
       'location': {'address': 'Binsey Walk',
        'lat': 51.50038136629462,
        'lng': 0.12501239776611328,
        'labeledLatLngs': [

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
#Cleaning Json and putting into pandas DF structure
venues = results['response']['groups'][0]['items']
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4e4a430e1850545dfc0289ae',
   'name': 'Southmere Lake',
   'location': {'address': 'Binsey Walk',
    'lat': 51.50038136629462,
    'lng': 0.12501239776611328,
    'labeledLatLngs': [{'label': 'display',
      'lat': 51.50038136629462,
      'lng': 0.12501239776611328}],
    'distance': 97,
    'postalCode': 'SE2 9TU',
    'cc': 'GB',
    'city': 'London',
    'state': 'Greater London',
    'country': 'United Kingdom',
    'formattedAddress': ['Binsey Walk',
     'London',
     'Greater London',
     'SE2 9TU',
     'United Kingdom']},
   'categories': [{'id': '4bf58dd8d48988d161941735',
     'name': 'Lake',
     'pluralName': 'Lakes',
     'shortName': 'Lake',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/lake_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, '

In [29]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4e4a430e1850545dfc0289ae-0,0,"[{'summary': 'This spot is popular', 'type': '...",4e4a430e1850545dfc0289ae,Southmere Lake,Binsey Walk,51.500381,0.125012,"[{'label': 'display', 'lat': 51.50038136629462...",97,SE2 9TU,GB,London,Greater London,United Kingdom,"[Binsey Walk, London, Greater London, SE2 9TU,...","[{'id': '4bf58dd8d48988d161941735', 'name': 'L...",0,[]
1,e-0-58246ba78d16903fc4e9235e-1,0,"[{'summary': 'This spot is popular', 'type': '...",58246ba78d16903fc4e9235e,G & B Roofing & Building,105 Ampleforth Rd,51.499117,0.117503,"[{'label': 'display', 'lat': 51.4991168, 'lng'...",459,SE2 9BG,GB,London,Greater London,United Kingdom,"[105 Ampleforth Rd, London, Greater London, SE...","[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",0,[]


In [30]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Southmere Lake,"[{'id': '4bf58dd8d48988d161941735', 'name': 'L...",51.500381,0.125012
1,G & B Roofing & Building,"[{'id': '5454144b498ec1f095bff2f2', 'name': 'C...",51.499117,0.117503


In [31]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Southmere Lake,Lake,51.500381,0.125012
1,G & B Roofing & Building,Construction & Landscaping,51.499117,0.117503


In [32]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.columns

Index(['name', 'categories', 'lat', 'lng'], dtype='object')

In [33]:
nearby_venues.head()

,name,categories,lat,lng
0,Southmere Lake,Lake,51.500381,0.125012
1,G & B Roofing & Building,Construction & Landscaping,51.499117,0.117503


In [34]:
#Creating a function to repeat the same for all neighborhoods in London  dataset
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 50 # limit of number of venues returned by Foursquare API
        radius = 500 # define radius
        CLIENT_ID='5I3DLW5XVAABDTQF2J2XRTHXWPANUUNZUUJDXXUJWB251M5D'
        CLIENT_SECRET='41VU5CVD1NFCUKDDKCOF4UDN0FTFOUGIJ0RJYOWTDTNOOMWC'
        VERSION='20180605'       
    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
lon_venues = getNearbyVenues(names=df_lon['neighborhood'],
                                   latitudes=df_lon['latitude'],
                                   longitudes=df_lon['longitude']
                                  )

Abbey Wood
Acton
Aldgate
Aldwych
Anerley
Angel
Archway
Arnos Grove
Balham
Bankside
Barbican
Barnes
Barnsbury
Battersea
Bayswater
Bedford Park
Belgravia
Bellingham
Belsize Park
Bermondsey
Bethnal Green
Blackfriars
Blackheath
Blackheath Royal Standard
Blackwall
Bloomsbury
Bounds Green
Bow
Bowes Park
Brent Cross
Brent Park
Brixton
Brockley
Bromley (also Bromley-by-Bow)
Brompton
Brondesbury
Brunswick Park
Burroughs, The
Camberwell
Cambridge Heath
Camden Town
Canary Wharf
Cann Hall
Canning Town
Canonbury
Castelnau
Catford
Chalk Farm
Charing Cross
Charlton
Chelsea
Childs Hill
Chinatown
Chinbrook
Chingford
Chiswick
Church End
Church End
Clapham
Clerkenwell
Colindale
Colliers Wood
Colney Hatch
Covent Garden
Cricklewood
Crofton Park
Crossness
Crouch End
Crystal Palace
Cubitt Town
Custom House
Dalston
De Beauvoir Town
Denmark Hill
Deptford
Dollis Hill
Dulwich
Ealing
Earls Court
Earlsfield
East Dulwich
East Finchley
East Ham
East Sheen
Edmonton
Elephant and Castle
Eltham
Farringdon
Finchley
Finsb

In [36]:
#Seeing how many venues we get per neighborhood
lon_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abbey Wood,2,2,2,2,2,2
Acton,50,50,50,50,50,50
Aldgate,50,50,50,50,50,50
Aldwych,48,48,48,48,48,48
Anerley,50,50,50,50,50,50


In [37]:
# one hot encoding
lon_onehot = pd.get_dummies(lon_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lon_onehot['Neighborhood'] = lon_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [lon_onehot.columns[-1]] + list(lon_onehot.columns[:-1])
lon_onehot = lon_onehot[fixed_columns]

lon_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Carpet Store,Caucasian Restaurant,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus School,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Gelato Shop,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Herbs & Spices Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lighting Store,Liquor Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Organic Grocery,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Repair,Shoe Store,Skate Park,Snack Place,Soccer Stadium,Social Club,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track Stadium,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,

In [38]:
#Grouping the rows by neighborhood and taking the mean of the frequency of the occurance
lon_grouped = lon_onehot.groupby('Neighborhood').mean().reset_index()
lon_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Campground,Canal,Candy Store,Cantonese Restaurant,Carpet Store,Caucasian Restaurant,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus School,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Gelato Shop,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Herbs & Spices Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lighting Store,Liquor Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Organic Grocery,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Repair,Shoe Store,Skate Park,Snack Place,Soccer Stadium,Social Club,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track Stadium,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop
0,Abbey Wood,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.00,0.000000,0.00,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.5,0.0

In [69]:
lon_grouped.drop(lon_grouped.columns[lon_grouped.columns.str.contains('Station')], inplace=True,axis=1)
lon_grouped.head()

,Neighborhood,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Garage,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Bike Rental / Bike Share,Bistro,Boat or Ferry,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Campground,Canal,Cantonese Restaurant,Caucasian Restaurant,Chaat Place,Champagne Bar,Chinese Restaurant,Church,Circus School,Cocktail Bar,College Cafeteria,College Quad,Comedy Club,Concert Hall,Construction & Landscaping,Convention Center,Creperie,Cricket Ground,Deli / Bodega,Diner,Distillery,Dive Bar,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Fishing Spot,Flea Market,Food,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Greek Restaurant,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Himalayan Restaurant,Historic Site,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hunan Restaurant,IT Services,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lounge,Malay Restaurant,Mamak Restaurant,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Organic Grocery,Outdoors & Recreation,Pakistani Restaurant,Park,Pedestrian Plaza,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Recreation Center,Restaurant,Rock Climbing Spot,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Repair,Skate Park,Snack Place,Soccer Stadium,Social Club,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Stables,Stadium,Steakhouse,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Tiki Bar,Track Stadium,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Abbey Wood,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.00,0.0,0.0000,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.00,0.5,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00
1,Acton,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.02,0.000000,0.02,0.0,0.0200,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.02,0.0,0.020000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.02,0.0,0.0,0.06,0.0,0.000000,0.00,0.00,0.02,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.06,0.00,0.0,0.0

In [70]:
#Writing a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [72]:
#Creating a new DF to display the top venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = lon_grouped['Neighborhood']

for ind in np.arange(lon_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lon_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey Wood,Lake,Construction & Landscaping,American Restaurant,Park,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant
1,Acton,Hotel,Garden,Wine Bar,Restaurant,Theater,Plaza,Pakistani Restaurant,Fountain,Pub,North Indian Restaurant
2,Aldgate,Café,Pub,Thai Restaurant,Indian Restaurant,Cocktail Bar,Italian Restaurant,Park,Ethiopian Restaurant,Vietnamese Restaurant,Escape Room
3,Aldwych,Pub,Bakery,Park,Café,Gastropub,Cocktail Bar,Music Venue,South American Restaurant,Spa,Middle Eastern Restaurant
4,Anerley,Hotel,Bar,Sandwich Place,Korean Restaurant,Café,Park,Steakhouse,Mexican Restaurant,Skate Park,Scenic Lookout


### 4. Cluster Neighborhoods

In [74]:
lon_grouped.drop_duplicates(subset ="Neighborhood", keep=False,inplace=True) 

In [76]:
# set number of clusters
kclusters = 7

lon_grouped_clustering = lon_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lon_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 2, 1, 4, 2, 1, 2, 2, 2, 1], dtype=int32)

In [77]:
#creating a new dataframe that includes cluster and the top 10 venues per neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [79]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,5,Abbey Wood,Lake,Construction & Landscaping,American Restaurant,Park,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant
1,2,Acton,Hotel,Garden,Wine Bar,Restaurant,Theater,Plaza,Pakistani Restaurant,Fountain,Pub,North Indian Restaurant
2,1,Aldgate,Café,Pub,Thai Restaurant,Indian Restaurant,Cocktail Bar,Italian Restaurant,Park,Ethiopian Restaurant,Vietnamese Restaurant,Escape Room
3,4,Aldwych,Pub,Bakery,Park,Café,Gastropub,Cocktail Bar,Music Venue,South American Restaurant,Spa,Middle Eastern Restaurant
4,2,Anerley,Hotel,Bar,Sandwich Place,Korean Restaurant,Café,Park,Steakhouse,Mexican Restaurant,Skate Park,Scenic Lookout


In [80]:
df_lon.rename(columns = {'neighborhood': 'Neighborhood'}, inplace = True)
df_lon.columns

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Index(['Neighborhood', 'borough', 'posttown', 'postcode', 'latitude',
       'longitude'],
      dtype='object')

In [85]:
# create map for London
map_clusters_lon = folium.Map(location=[lat_lon, lng_lon], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_lon['latitude'], df_lon['longitude'], df_lon['Neighborhood'], df_lon['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(map_clusters_lon)
       
map_clusters_lon

### 5. Examine Clusters

#### Cluster 1

In [88]:
df_lon.loc[df_lon['Cluster Labels'] == 0, df_lon.columns[[0] + [1]+ [4] + list(range(5, df_lon.shape[1] ))]]

,Neighborhood,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,Chalk Farm,Camden,51.433603,0.039923,0,Gym / Fitness Center,American Restaurant,Outdoors & Recreation,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant
173,Fitzrovia,Camden,51.433603,0.039923,0,Gym / Fitness Center,American Restaurant,Outdoors & Recreation,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant
251,Hoxton,Hackney,51.431180,-0.178316,0,Gym / Fitness Center,Playground,Brewery,Supermarket,American Restaurant,Organic Grocery,Music Venue,New American Restaurant,Newsagent,Nightclub
252,The Hyde,Barnet,51.431180,-0.178316,0,Gym / Fitness Center,Playground,Brewery,Supermarket,American Restaurant,Organic Grocery,Music Venue,New American Restaurant,Newsagent,Nightclub
273,Ladywell,Lewisham,51.433603,0.039923,0,Gym / Fitness Center,American Restaurant,Outdoors & Recreation,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant


#### Cluster 2

In [89]:
df_lon.loc[df_lon['Cluster Labels'] == 1, df_lon.columns[[0] + [1]+ [4] + list(range(5, df_lon.shape[1]))]]

,Neighborhood,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Aldgate,City,51.532760,-0.114007,1,Café,Pub,Thai Restaurant,Indian Restaurant,Cocktail Bar,Italian Restaurant,Park,Ethiopian Restaurant,Vietnamese Restaurant,Escape Room
10,Angel,Islington,51.530130,-0.107969,1,Pub,Café,Indian Restaurant,Pizza Place,Cocktail Bar,Gym / Fitness Center,Korean Restaurant,Nightclub,Middle Eastern Restaurant,Hotel
17,Bankside,Southwark,51.439820,0.000393,1,Pharmacy,Construction & Landscaping,American Restaurant,Pakistani Restaurant,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant
34,Bedford Park,Ealing,51.498105,-0.165170,1,Café,Japanese Restaurant,Italian Restaurant,Hotel,Steakhouse,Lebanese Restaurant,Indian Restaurant,Hookah Bar,Health & Beauty Service,Pharmacy
41,Bermondsey,Southwark,51.522385,-0.045831,1,Pub,Gym / Fitness Center,Gym,Thai Restaurant,Supermarket,Noodle House,Chinese Restaurant,Park,Café,Bar
43,Bethnal Green,Tower Hamlets,51.519895,-0.075422,1,Pub,Wine Bar,Market,Indian Restaurant,Middle Eastern Restaurant,French Restaurant,BBQ Joint,Flea Market,Speakeasy,Chinese Restaurant
49,Blackfriars,City,51.485126,0.029115,1,Supermarket,American Restaurant,Soccer Stadium,Pub,Chinese Restaurant,Gym / Fitness Center,Thai Restaurant,Organic Grocery,New American Restaurant,Newsagent
50,Blackheath,Lewisham,51.498105,-0.165170,1,Café,Japanese Restaurant,Italian Restaurant,Hotel,Steakhouse,Lebanese Restaurant,Indian Restaurant,Hookah Bar,Health & Beauty Service,Pharmacy
56,Bounds Green,Haringey,51.537720,-0.136485,1,Pub,Pizza Place,Italian Restaurant,Greek Restaurant,Restaurant,Tea Room,Pharmacy,Beer Bar,Garden,Rock Club
58,Bowes Park,Haringey,51.465705,-0.144997,1,Pub,Gym / Fitness Center,French Restaurant,Restaurant,Bistro,Bar,Gastropub,Breakfast Spot,Hotel,Deli / Bodega


#### Cluster 3

In [90]:
df_lon.loc[df_lon['Cluster Labels'] == 2, df_lon.columns[[0] + [1]+ [4] + list(range(5, df_lon.shape[1]))]]

,Neighborhood,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Acton,Ealing,51.507408,-0.127699,2,Hotel,Garden,Wine Bar,Restaurant,Theater,Plaza,Pakistani Restaurant,Fountain,Pub,North Indian Restaurant
9,Anerley,Bromley,51.500212,-0.115440,2,Hotel,Bar,Sandwich Place,Korean Restaurant,Café,Park,Steakhouse,Mexican Restaurant,Skate Park,Scenic Lookout
12,Archway,Islington,51.507408,-0.127699,2,Hotel,Garden,Wine Bar,Restaurant,Theater,Plaza,Pakistani Restaurant,Fountain,Pub,North Indian Restaurant
15,Arnos Grove,Enfield,51.507408,-0.127699,2,Hotel,Garden,Wine Bar,Restaurant,Theater,Plaza,Pakistani Restaurant,Fountain,Pub,North Indian Restaurant
16,Balham,Wandsworth,51.507408,-0.127699,2,Hotel,Garden,Wine Bar,Restaurant,Theater,Plaza,Pakistani Restaurant,Fountain,Pub,North Indian Restaurant
27,Battersea,Wandsworth,51.507408,-0.127699,2,Hotel,Garden,Wine Bar,Restaurant,Theater,Plaza,Pakistani Restaurant,Fountain,Pub,North Indian Restaurant
39,Belsize Park,Camden,51.507408,-0.127699,2,Hotel,Garden,Wine Bar,Restaurant,Theater,Plaza,Pakistani Restaurant,Fountain,Pub,North Indian Restaurant
52,Blackwall,Tower Hamlets,51.507408,-0.127699,2,Hotel,Garden,Wine Bar,Restaurant,Theater,Plaza,Pakistani Restaurant,Fountain,Pub,North Indian Restaurant
54,Bloomsbury,Camden,51.507408,-0.127699,2,Hotel,Garden,Wine Bar,Restaurant,Theater,Plaza,Pakistani Restaurant,Fountain,Pub,North Indian Restaurant
57,Bow,Tower Hamlets,51.507408,-0.127699,2,Hotel,Garden,Wine Bar,Restaurant,Theater,Plaza,Pakistani Restaurant,Fountain,Pub,North Indian Restaurant


#### Cluster 4

In [91]:
df_lon.loc[df_lon['Cluster Labels'] == 3, df_lon.columns[[0] + [1]+ [4] + list(range(5, df_lon.shape[1]))]]

,Neighborhood,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Blackheath Royal Standard,Greenwich,51.566830,-0.214939,3,Garden,Pizza Place,Park,Organic Grocery,Multiplex,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House
60,Brent Cross,Barnet,51.604496,-0.242665,3,Park,Pakistani Restaurant,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Organic Grocery
64,Brockley,Lewisham,51.566830,-0.214939,3,Garden,Pizza Place,Park,Organic Grocery,Multiplex,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House
75,Cambridge Heath,Tower Hamlets,51.566830,-0.214939,3,Garden,Pizza Place,Park,Organic Grocery,Multiplex,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House
133,De Beauvoir Town,Islington,51.604496,-0.242665,3,Park,Pakistani Restaurant,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Organic Grocery
143,Earlsfield,Wandsworth,51.604496,-0.242665,3,Park,Pakistani Restaurant,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant,Organic Grocery
175,Forest Gate,Newham,51.566830,-0.214939,3,Garden,Pizza Place,Park,Organic Grocery,Multiplex,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House
181,Frognal,Camden,51.418520,-0.116618,3,Café,Garden,Brewery,Park,American Restaurant,Outdoors & Recreation,New American Restaurant,Newsagent,Nightclub,Noodle House
244,Homerton,Hackney,51.418520,-0.116618,3,Café,Garden,Brewery,Park,American Restaurant,Outdoors & Recreation,New American Restaurant,Newsagent,Nightclub,Noodle House


#### Cluster 5

In [93]:
df_lon.loc[df_lon['Cluster Labels'] == 4, df_lon.columns[[0] + [1]+ [4] + list(range(5, df_lon.shape[1]))]]

,Neighborhood,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Aldwych,Westminster,51.542690,-0.109138,4,Pub,Bakery,Park,Café,Gastropub,Cocktail Bar,Music Venue,South American Restaurant,Spa,Middle Eastern Restaurant
18,Barbican,City,51.560430,-0.172355,4,Lake,Pub,Italian Restaurant,Outdoors & Recreation,Gastropub,Café,Diner,Indian Restaurant,Monument / Landmark,Fishing Spot
22,Barnes,Richmond upon Thames,51.478275,0.014093,4,Pub,Chinese Restaurant,Park,Café,Juice Bar,Japanese Restaurant,Nightclub,Noodle House,Jazz Club,North Indian Restaurant
26,Barnsbury,Islington,51.542690,-0.109138,4,Pub,Bakery,Park,Café,Gastropub,Cocktail Bar,Music Venue,South American Restaurant,Spa,Middle Eastern Restaurant
28,Bayswater,Westminster,51.602440,-0.102554,4,Garden Center,Café,Pub,Track Stadium,Stadium,Park,Food,Organic Grocery,Music Venue,New American Restaurant
35,Belgravia,Westminster,51.533085,-0.195979,4,Park,New American Restaurant,Pub,Newsagent,Recreation Center,Middle Eastern Restaurant,Hotel,Lounge,Outdoors & Recreation,Nightclub
36,Bellingham,Lewisham,51.542690,-0.109138,4,Pub,Bakery,Park,Café,Gastropub,Cocktail Bar,Music Venue,South American Restaurant,Spa,Middle Eastern Restaurant
74,Camberwell,Southwark,51.485760,-0.029205,4,Park,Pub,Vietnamese Restaurant,Playground,Fast Food Restaurant,Outdoors & Recreation,Music Venue,New American Restaurant,Newsagent,Nightclub
76,Camden Town,Camden,51.438115,-0.091826,4,Park,Pub,Bakery,Pakistani Restaurant,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant
79,Canning Town,Newham,51.458806,-0.188133,4,Pub,Gym / Fitness Center,Bakery,Chaat Place,Movie Theater,Café,Breakfast Spot,Sandwich Place,Hotel,Thai Restaurant


#### Cluster 6

In [94]:
df_lon.loc[df_lon['Cluster Labels'] == 5, df_lon.columns[[0] + [1]+ [4] + list(range(5, df_lon.shape[1]))]]

,Neighborhood,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey Wood,Bexley,51.499741,0.124061,5,Lake,Construction & Landscaping,American Restaurant,Park,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant
66,Bromley (also Bromley-by-Bow),Tower Hamlets,51.499741,0.124061,5,Lake,Construction & Landscaping,American Restaurant,Park,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant
280,Lee,Lewisham,51.499741,0.124061,5,Lake,Construction & Landscaping,American Restaurant,Park,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant


#### Cluster 7

In [96]:
df_lon.loc[df_lon['Cluster Labels'] == 6, df_lon.columns[[0] + [1]+ [4] + list(range(5, df_lon.shape[1]))]]

,Neighborhood,borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
259,Kennington,Lambeth,51.458949,-0.102433,6,Park,Bar,Pakistani Restaurant,Music Venue,New American Restaurant,Newsagent,Nightclub,Noodle House,North Indian Restaurant,Okonomiyaki Restaurant


### Conclusion

It shows that Cluster 2 has the more Asian Restaurants found. To start up an Asian catering business, it's recommended to start in clusters other than cluseter 2. 